In [205]:
from utils_filter import *
from utils_datetime import *
from dateutil import parser
from dateutil.relativedelta import relativedelta

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [206]:
data_location = 'data'
moderate = False # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [207]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [208]:
# add dates to reports and pph in same format as in outlooks
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) #TODO: or end time?
pph['time'] = get_pph_date_strings(pph.time) 

In [212]:
# TODO: subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks
outlooks[outlooks['CATEGORY'] == 'CATEGORICAL'] # use cycle to select just one day 1?

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry
0,198701011200,198701021200,198701010635,C,1,TSTM,CATEGORICAL,-1,198701010000,"MULTIPOLYGON (((-80.69500 29.36500, -80.68200 ..."
1,198701011500,198701021200,198701011441,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-70.15100 42.90900, -70.14600 42.899..."
2,198701011900,198701021200,198701011849,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-73.73200 39.90700, -73.73200 39.892..."
3,198701021200,198701031200,198701020637,C,1,TSTM,CATEGORICAL,-1,198701020000,"POLYGON ((-124.43300 41.91800, -124.44100 41.9..."
5,198701021500,198701031200,198701021438,C,1,TSTM,CATEGORICAL,-1,198701020000,"MULTIPOLYGON (((-89.75800 28.96700, -89.75900 ..."
...,...,...,...,...,...,...,...,...,...,...
183166,202312291630,202312301200,202312291601,C,1,TSTM,CATEGORICAL,16,202312290000,"POLYGON ((-121.88000 34.66000, -122.05900 34.8..."
183167,202312301200,202312311200,202312291644,C,2,TSTM,CATEGORICAL,17,202312300000,"POLYGON ((-119.53500 33.98700, -120.06600 33.9..."
183168,202312292000,202312301200,202312291927,C,1,TSTM,CATEGORICAL,20,202312290000,"POLYGON ((-121.88000 34.66000, -122.05900 34.8..."
183169,202312300100,202312301200,202312300042,C,1,TSTM,CATEGORICAL,1,202312290000,"POLYGON ((-121.88000 34.66000, -122.05900 34.8..."


In [220]:
def add_outlooks_label(outlooks, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding a new column in outlooks")
    outlooks[label_name] = none_label
    for label, dates in zip(labels, label_dates):
        #print(label)
        outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
    return outlooks

def add_pph_label(pph, label_dates, labels, label_name, none_label):
    # adds new variable with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding new variable in pph")
    pph[label_name] = (('time'), np.full(len(pph['time']), none_label))
    for label, dates in zip(labels, label_dates):
        #print(label)
        pph[label_name].loc[pph['time'].isin(dates)] = label  
    return pph

def add_reports_label(reports, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    reports[label_name] = none_label
    print("adding a new column in reports")
    for label, dates in zip(labels, label_dates):
       #print(label)
       reports[label_name].loc[reports['DATE'].isin(dates)] = label # TODO: syntax may be wrong
    return reports

def add_labels(outlooks, pph, reports, label_dates, labels, label_name, none_label):
    # adds labels, overwriting with later ones if a date has multiple labels
    return(add_outlooks_label(outlooks, label_dates, labels, label_name, none_label), 
           add_pph_label(pph, label_dates, labels, label_name, none_label),
           add_reports_label(reports, label_dates, labels, label_name, none_label))



Subset by threshold

In [222]:
# add max threshold forecasted for valid day to each datapoint

categories = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']
category_dates = []
for category in categories:
    category_dates.append(identify_dates_above_threshold(outlooks, category))

(new_outlooks, new_pph, new_reports) = add_labels(outlooks, pph, reports, category_dates, categories, 'Categorical Risk', 'NONE')
new_reports


C:\Users\miles\AppData\Local\Temp\ipykernel_1080\2380181307.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label


adding new variable in pph
TSTM
MRGL
SLGT
ENH
MDT
HIGH
TSTM
MRGL
SLGT


C:\Users\miles\AppData\Local\Temp\ipykernel_1080\2380181307.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports[label_name].loc[reports['DATE'].isin(dates)] = label # TODO: syntax may be wrong


ENH
MDT
HIGH


,field_1,STATE,EVENT_TYPE,CZ_TYPE,CZ_NAME,WFO,BEGIN_DATE_TIME,CZ_TIMEZONE,END_DATE_TIME,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,DATE,geometry,Categorical Risk
0,0,OKLAHOMA,Tornado,C,WASHITA,,28-APR-50 14:45:00,CST,28-APR-50 14:45:00,0,...,0,,,35.12,-99.20,35.17,-99.20,195004280000,None,NONE
1,1,TEXAS,Tornado,C,COMANCHE,,29-APR-50 15:30:00,CST,29-APR-50 15:30:00,0,...,0,,,31.90,-98.60,31.73,-98.60,195004290000,None,NONE
2,2,PENNSYLVANIA,Tornado,C,LEHIGH,,05-JUL-50 18:00:00,CST,05-JUL-50 18:00:00,2,...,0,,,40.58,-75.70,40.65,-75.47,195007050000,None,NONE
3,3,PENNSYLVANIA,Tornado,C,DAUPHIN,,05-JUL-50 18:30:00,CST,05-JUL-50 18:30:00,0,...,0,,,40.60,-76.75,,,195007050000,None,NONE
4,4,PENNSYLVANIA,Tornado,C,CRAWFORD,,24-JUL-50 14:40:00,CST,24-JUL-50 14:40:00,0,...,0,,,41.63,-79.68,,,195007240000,None,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054186,72123,ARIZONA,Thunderstorm Wind,C,PIMA,TWC,31-JUL-23 18:40:00,MST-7,31-JUL-23 18:40:00,0,...,5,WSW,(TUS)TUCSON INTL ARP,32.0903,-111.0226,32.0903,-111.0226,202307310000,None,SLGT
1054187,72124,ARIZONA,Thunderstorm Wind,C,PIMA,TWC,31-JUL-23 19:35:00,MST-7,31-JUL-23 19:35:00,0,...,1,S,TUSCON MARANA NW AIRPORT,32.4,-111.22,32.4,-111.22,202307310000,None,SLGT
1054188,72125,GEORGIA,Thunderstorm Wind,C,DOUGLAS,FFC,06-AUG-23 13:25:00,EST-5,06-AUG-23 13:27:00,0,...,2,NNW,CHAPEL HILL,33.7,-84.73,33.7,-84.73,202308060000,None,SLGT
1054189,72132,MINNESOTA,Tornado,C,NORMAN,FGF,24-JUN-23 15:10:00,CST-6,24-JUN-23 15:14:00,0,...,4,ENE,SHELLY,47.49,-96.73,47.4991,-96.7571,202306240000,None,SLGT


Subset by ramp up/down

In [ ]:
# define and add ramp category for each datapoint. Potentially add 2 binary ramp up and ramp down (4 options are [up, down, up and down, niether]). How many forecasts to consider for each day? The day 3, both day 2, and the first day 1 (so 4 forecasts ramp)

Subset by accurate/inaccurate

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time. Decide if we should denote these somehow..

Subset by season

In [71]:
# add column denoting season (4 met seasons as starting point)

'198701010000'

Subset by region

In [ ]:
# decide how to do... lower priotity, could do by state of center of highest category (or center of PPH?) Could start with super basic version

Subset by environmental data (to do later)

In [ ]:
# resave labelled data
# TODO: create master function that subsets data based on easy inputs once it's all been labelled. Put in a util file